# Task 5: Strategy Backtesting

### 1. Define Backtesting Period and Benchmark

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Load historical data (from Task 1 CSVs)
prices = pd.DataFrame({
    'TSLA': pd.read_csv('data/TSLA_historical.csv', index_col='Date', parse_dates=True)['Adj Close'],
    'BND': pd.read_csv('data/BND_historical.csv', index_col='Date', parse_dates=True)['Adj Close'],
    'SPY': pd.read_csv('data/SPY_historical.csv', index_col='Date', parse_dates=True)['Adj Close']
}).dropna()

# Backtesting period
backtest_start = '2024-08-01'
backtest_end = '2025-07-31'
backtest_prices = prices.loc[backtest_start:backtest_end]

# Portfolio weights (from Task 4, replace with your actual weights)
strategy_weights = {'TSLA': 0.3, 'BND': 0.5,
                    'SPY': 0.2}  # Example Max Sharpe weights
benchmark_weights = {'TSLA': 0.0, 'BND': 0.4, 'SPY': 0.6}

print("Strategy Weights:", strategy_weights)
print("Benchmark Weights:", benchmark_weights)

### 2. Simulate Portfolio Performance

In [ ]:
# Daily returns
daily_returns = backtest_prices.pct_change().dropna()

# Portfolio daily returns
strategy_daily = sum(
    strategy_weights[ticker] * daily_returns[ticker] for ticker in strategy_weights)
benchmark_daily = sum(
    benchmark_weights[ticker] * daily_returns[ticker] for ticker in benchmark_weights)

# Cumulative returns
strategy_cumulative = (1 + strategy_daily).cumprod() - 1
benchmark_cumulative = (1 + benchmark_daily).cumprod() - 1

### 3. Analyze Performance

- Compute total return and annualized Sharpe Ratio (use 2% risk-free rate, as assumed in Task 4).
- Plot cumulative returns for visualization.

In [ ]:
# Performance metrics
def calculate_metrics(daily_returns, risk_free_rate=0.02):
    total_return = (1 + daily_returns).prod() - 1
    sharpe_ratio = (daily_returns.mean() * 252 - risk_free_rate) / \
        (daily_returns.std() * np.sqrt(252))
    return total_return, sharpe_ratio


strategy_total, strategy_sharpe = calculate_metrics(strategy_daily)
benchmark_total, benchmark_sharpe = calculate_metrics(benchmark_daily)

print(
    f"Strategy - Total Return: {strategy_total:.2%}, Sharpe Ratio: {strategy_sharpe:.2f}")
print(
    f"Benchmark - Total Return: {benchmark_total:.2%}, Sharpe Ratio: {benchmark_sharpe:.2f}")

# Plot cumulative returns
plt.figure(figsize=(10, 6))
plt.plot(strategy_cumulative, label='Strategy Portfolio')
plt.plot(benchmark_cumulative, label='Benchmark (60% SPY, 40% BND)')
plt.title('Cumulative Returns: Strategy vs Benchmark (Aug 2024 - Jul 2025)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.savefig('reports/backtest_cumulative_returns.png')
plt.show()